In [211]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

In [122]:
def cleanData(X):
    X.Sex = X.Sex.replace(["female", "male"], [1, 0])
    X.Embarked = X.Embarked.replace(["C", "Q", "S"], [3, 2, 1])


In [123]:
def maeCalc(X_train, X_test, y_train, y_test, maxLeaves):
    ##decision tree regressor model
    decisionModel = DecisionTreeRegressor(random_state = 1, max_leaf_nodes = maxLeaves)
    decisionModel.fit(X_train,y_train)
    decisionPredicted = decisionModel.predict(X_test)
    decisionError = mean_absolute_error(decisionPredicted, y_test)
    
    ##random forest regressor model
    forestModel = RandomForestRegressor(random_state = 1, max_leaf_nodes = maxLeaves)
    forestModel.fit(X_train,y_train)
    forestPredicted = forestModel.predict(X_test)
    forestError = mean_absolute_error(forestPredicted, y_test)

    return [decisionError, forestError]

In [204]:
##cleaning data
rawData = pd.read_csv('titanic/train.csv')
#rawData.describe()
rawData.isnull().sum()
features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked','Age']
X = rawData[features]
cleanData(X)

y = rawData['Survived']
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)
##impute
imp = SimpleImputer(missing_values = np.nan, strategy='mean')
X_train = pd.DataFrame(imp.fit_transform(X_train))
X_train.columns = X_test.columns
X_test = pd.DataFrame(imp.transform(X_test))
X_test.columns = X_train.columns

X_train.head()
#y_train.head()

C:\Users\darin\AppData\Local\Temp\ipykernel_61004\3842542810.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.Sex = X.Sex.replace(["female", "male"], [1, 0])
C:\Users\darin\AppData\Local\Temp\ipykernel_61004\3842542810.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.Embarked = X.Embarked.replace(["C", "Q", "S"], [3, 2, 1])


,Pclass,Sex,SibSp,Parch,Fare,Embarked,Age
0,1.0,0.0,1.0,0.0,52.0000,1.0,42.000000
1,3.0,0.0,1.0,0.0,15.5000,2.0,30.073682
2,1.0,0.0,1.0,0.0,89.1042,3.0,49.000000
3,1.0,1.0,1.0,0.0,91.0792,3.0,19.000000
4,1.0,0.0,1.0,0.0,53.1000,1.0,19.000000


In [202]:
leafArray = range(10, 400)  
dTreeError = {}
rTreeError = {}
for maxLeaves in leafArray:
    outputList = maeCalc(X_train, X_test, y_train, y_test, maxLeaves)
    dTreeError[maxLeaves] = outputList[0]
    rTreeError[maxLeaves] = outputList[1]


In [208]:
dTreeErrorMinIndex = min(dTreeError, key=dTreeError.get)
rTreeErrorMinIndex = min(rTreeError, key=rTreeError.get)
print("Decision Tree min mae - Max Leaves: {} MAE: {:.4f}".format(dTreeErrorMinIndex, min(dTreeError.values())))
print("Random Forest min mae - Max Leaves: {} MAE: {:.4f}".format(rTreeErrorMinIndex, min(rTreeError.values())))


Decision Tree min mae - Max Leaves: 52 MAE: 0.2318
Random Forest min mae - Max Leaves: 48 MAE: 0.2515


In [213]:
#Test

#model = DecisionTreeRegressor(random_state = 1, max_leaf_nodes = dTreeErrorMinIndex)
#model = RandomForestRegressor(random_state = 1, max_leaf_nodes = rTreeErrorMinIndex)
model = LogisticRegression(random_state = 1)

model.fit(X_train,y_train)
finalData = pd.read_csv('titanic/test.csv') #6873
X = finalData[features]
cleanData(X)
#X.Fare = X.Fare.replace(np.nan, 1000)
X = pd.DataFrame(imp.transform(X))
X.columns = X_test.columns
ans = model.predict(X)
ans = [round(num) for num in ans]
output = pd.DataFrame({'PassengerId': finalData.PassengerId, 'Survived': ans})
output.to_csv('ans.csv', index=False)

c:\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\darin\AppData\Local\Temp\ipykernel_61004\3842542810.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.Sex = X.Sex.replace(["female", "male"], [1, 0])
C:\Users\darin\AppData\Local\Temp\ipykernel_61004\3842542810.py:3: SettingWithCopyWarning: 